# ĐỒ ÁN CUỐI KỲ - PHÂN TÍCH DỮ LIỆU BẢO TOÀN TÍNH RIÊNG TƯ

## THÀNH VIÊN NHÓM:

- 22120304 - Nguyễn Thị Kim Quý

- 22120338 - Đỗ Hạnh Thảo

- 22120352 - Phạm Nguyễn Quang Thoại

In [2]:
import pandas as pd
import numpy as np
import re

### 1. Đọc file

In [11]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

print("Số dòng trong train.csv", len(train_data))
print(f"\nSố cột: {len(train_data.columns)}")
print("Tên các cột:", list(train_data.columns))
print("\n 5 dòng đầu trong train.csv:\n")
print(train_data.head())

Số dòng trong train.csv 1317

Số cột: 20
Tên các cột: ['Make', 'Model', 'Price', 'Year', 'Kilometer', 'Fuel Type', 'Transmission', 'Location', 'Color', 'Owner', 'Seller Type', 'Engine', 'Max Power', 'Max Torque', 'Drivetrain', 'Length', 'Width', 'Height', 'Seating Capacity', 'Fuel Tank Capacity']

 5 dòng đầu trong train.csv:

        Make                                    Model    Price  Year  \
0  Ssangyong                               Rexton RX7   975000  2013   
1    Hyundai  Creta SX (O) 1.5 Petrol CVT [2020-2022]  1748999  2022   
2       Audi                     A4 2.0 TDI (143 bhp)  1150000  2012   
3    Hyundai        Grand i10 Magna AT 1.2 Kappa VTVT   549000  2018   
4    Hyundai                       Elite i20 Asta 1.2   675000  2017   

   Kilometer Fuel Type Transmission   Location   Color   Owner Seller Type  \
0      72000    Diesel    Automatic  Bangalore  Silver   First  Individual   
1       2670    Petrol    Automatic    Kolkata   White   First  Individual   
2   

In [12]:
print("Số dòng trong test.csv", len(test_data))
print(f"\nSố cột: {len(test_data.columns)}")
print("Tên các cột:", list(test_data.columns))
print("\n 5 dòng đầu trong test.csv:\n")
print(test_data.head())

Số dòng trong test.csv 330

Số cột: 20
Tên các cột: ['Make', 'Model', 'Price', 'Year', 'Kilometer', 'Fuel Type', 'Transmission', 'Location', 'Color', 'Owner', 'Seller Type', 'Engine', 'Max Power', 'Max Torque', 'Drivetrain', 'Length', 'Width', 'Height', 'Seating Capacity', 'Fuel Tank Capacity']

 5 dòng đầu trong test.csv:

            Make                           Model    Price  Year  Kilometer  \
0           Ford      Endeavour Trend 2.2 4x2 AT  2350000  2016      75000   
1         Toyota  Urban Cruiser Premium Grade AT  1050000  2021       1910   
2  Maruti Suzuki                    Alto 800 Lxi   210000  2014      42505   
3          Honda                  City SV Diesel   550000  2014      85000   
4        Hyundai                       Eon Era +   290000  2018      70000   

  Fuel Type Transmission  Location   Color   Owner Seller Type   Engine  \
0    Diesel    Automatic    Mohali   White  Second  Individual  2198 cc   
1    Petrol    Automatic  Varanasi   White   First  Ind

### 2. Tiền xử lý dữ liệu

In [13]:
# Hàm trích xuất số từ chuỗi
def extract_number(s):
    if pd.isna(s):
        return np.nan
    match = re.search(r'(\d+)', str(s).replace(',', ''))
    if match:
        return int(match.group(1))
    return np.nan

# Hàm label encode
def label_encode(series, mapping=None):
    if mapping is None:
        uniques = series.dropna().unique()
        mapping = {k: v for v, k in enumerate(sorted(uniques))}
    encoded = series.map(mapping)
    return encoded, mapping

In [14]:
# Bước 1: Trích số từ các cột chuỗi số
for col in ['Engine', 'Max Power', 'Max Torque']:
    train_data[col] = train_data[col].apply(extract_number)
    test_data[col] = test_data[col].apply(extract_number)

# Bước 2: Label encoding thủ công cho các cột phân loại
categorical_cols = ['Make', 'Model', 'Fuel Type', 'Transmission', 'Location', 
                    'Color', 'Owner', 'Seller Type', 'Drivetrain']
mappings = {}

for col in categorical_cols:
    train_data[col], mappings[col] = label_encode(train_data[col])
    test_data[col] = test_data[col].map(mappings[col])

# Bước 3: Xử lý giá trị thiếu
# Với categorical NaN -> -1
for col in categorical_cols:
    train_data[col] = train_data[col].fillna(-1)
    test_data[col] = test_data[col].fillna(-1)

# Với numeric NaN -> trung bình cột train
numeric_cols = ['Year', 'Kilometer', 'Engine', 'Max Power', 'Max Torque', 
                'Length', 'Width', 'Height', 'Seating Capacity', 'Fuel Tank Capacity']
for col in numeric_cols:
    mean = train_data[col].mean()
    train_data[col] = train_data[col].fillna(mean)
    test_data[col] = test_data[col].fillna(mean)

# Bước 4: Chuẩn hóa dữ liệu numeric
for col in numeric_cols:
    mean = train_data[col].mean()
    std = train_data[col].std()
    train_data[col] = (train_data[col] - mean) / std
    test_data[col] = (test_data[col] - mean) / std

# Bước 5: Tách X và Y
y_train = train_data['Price']
X_train = train_data.drop(columns=['Price'])
y_test = test_data['Price']
X_test = test_data.drop(columns=['Price'])

print("Sau khi tiển xử lý dữ liệu cho train_data: ")
print("Train.shape = ", train_data.shape)
print("Các cột trong train_data:", list(train_data.columns))
print("\n 5 dòng đầu trong train.csv:\n")
print(train_data.head())


Sau khi tiển xử lý dữ liệu cho train_data: 
Train.shape =  (1317, 20)
Các cột trong train_data: ['Make', 'Model', 'Price', 'Year', 'Kilometer', 'Fuel Type', 'Transmission', 'Location', 'Color', 'Owner', 'Seller Type', 'Engine', 'Max Power', 'Max Torque', 'Drivetrain', 'Length', 'Width', 'Height', 'Seating Capacity', 'Fuel Tank Capacity']

 5 dòng đầu trong train.csv:

   Make  Model    Price      Year  Kilometer  Fuel Type  Transmission  \
0    25    538   975000 -0.989540   0.274506          1             0   
1     7    227  1748999  1.644419  -0.817523          5             0   
2     0     33  1150000 -1.282202  -0.229044          1             0   
3     7    397   549000  0.473770  -0.024767          5             0   
4     7    295   675000  0.181108  -0.094214          5             1   

   Location  Color  Owner  Seller Type        Engine     Max Power  \
0         6     14      0            2  1.503194e+00  8.338177e-01   
1        33     15      0            2 -3.225349e-

In [15]:
print("Sau khi tiền xử lý dữ liệu cho test_data:")
print("Test.shape = ", test_data.shape)
print("Các cột trong test_data:", list(test_data.columns))
print("\n 5 dòng đầu trong test.csv:\n")
print(test_data.head())

Sau khi tiền xử lý dữ liệu cho test_data:
Test.shape =  (330, 20)
Các cột trong test_data: ['Make', 'Model', 'Price', 'Year', 'Kilometer', 'Fuel Type', 'Transmission', 'Location', 'Color', 'Owner', 'Seller Type', 'Engine', 'Max Power', 'Max Torque', 'Drivetrain', 'Length', 'Width', 'Height', 'Seating Capacity', 'Fuel Tank Capacity']

 5 dòng đầu trong test.csv:

   Make  Model    Price      Year  Kilometer  Fuel Type  Transmission  \
0   5.0   -1.0  2350000 -0.111554   0.321759        1.0             0   
1  27.0   -1.0  1050000  1.351757  -0.829494        5.0             0   
2  17.0   56.0   210000 -0.696878  -0.190075        5.0             1   
3   6.0  172.0   550000 -0.696878   0.479271        1.0             1   
4   7.0  320.0   290000  0.473770   0.243003        5.0             1   

   Location  Color  Owner  Seller Type    Engine  Max Power  Max Torque  \
0      41.0     15    2.0            2  0.744884   0.430370    0.974486   
1      68.0     15    0.0            2 -0.3758

### 3.1 

### 3.2 

### 3.3